In [42]:
import pandas as pd
import csv
import numpy as np
from IPython.display import display, HTML

In [16]:
source_f = open('source.txt')
sources = [line.strip().split('\t') for line in source_f.readlines()]
sources = [(x, int(y)) for x, y in sources]

In [17]:
level_f = open('level.txt')
levels = [line.strip().split('\t') for line in level_f.readlines()]
levels = [(x, int(y)) for x, y in levels]

In [18]:
gt_f = open('gt.txt')
gts = [line.strip().split('\t') for line in gt_f.readlines()]

In [7]:
sources[:5]

[('train_00000.jpg', 1),
 ('train_00001.jpg', 0),
 ('train_00002.jpg', 0),
 ('train_00003.jpg', 1),
 ('train_00004.jpg', 0)]

In [8]:
gts[:5]

[['train_00000.jpg', '4 \\times 7 = 2 8'],
 ['train_00001.jpg', 'a ^ { x } > q'],
 ['train_00002.jpg', '8 \\times 9'],
 ['train_00003.jpg',
  '\\sum _ { k = 1 } ^ { n - 1 } b _ { k } = a _ { n } - a _ { 1 }'],
 ['train_00004.jpg', 'I = d q / d t']]

In [19]:
sources_df = pd.DataFrame(sources, columns=['path', 'source'])
gts_df = pd.DataFrame(gts, columns=['path', 'gt'])
levels_df = pd.DataFrame(levels, columns=['path', 'level'])

len(sources_df), len(gts_df), len(levels_df)

(100000, 100000, 100000)

In [20]:
merged = sources_df.merge(levels_df).merge(gts_df)

In [21]:
groups = merged.groupby(['source', 'level'])

In [14]:
for source in range(2):
    for level in range(1, 5 + 1):
        try:
            g = groups.get_group((source, level))
            g.loc[:, ['path', 'gt']].to_csv(
                f'gt_s:{source}_l:{level}.txt',
                sep='\t',
                header=False,
                index=False,
                quoting=csv.QUOTE_NONE,
            )
        except KeyError:
            print(f'Empty source: {source} level: {level}')

Empty source: 1 level: 5


In [101]:
groups.count()

path     gt
source level              
0      1       3748   3748
       2      32288  32288
       3       9236   9236
       4       4156   4156
       5        572    572
1      1       9753   9753
       2      11242  11242
       3      24548  24548
       4       4457   4457

In [102]:
import torch

In [103]:
device = torch.device('cuda')

In [105]:
device.type

'cuda'

In [41]:
# for source in range(2):
#     for level in range(1, 5 + 1):
#         try:
#             g = groups.get_group((source, level))
#             train_len = round(len(g) * (1-0.2))
#             indexes = np.arange(len(g))
#             np.random.shuffle(indexes)
#             train_idx = indexes[:train_len]
#             val_idx = indexes[train_len:]
            
#             g.iloc[train_idx].loc[:, ['path', 'gt']].to_csv(
#                 f'train_gt_s:{source}_l:{level}.txt',
#                 sep='\t',
#                 header=False,
#                 index=False,
#                 quoting=csv.QUOTE_NONE,
#             )
            
#             g.iloc[val_idx].loc[:, ['path', 'gt']].to_csv(
#                 f'test_gt_s:{source}_l:{level}.txt',
#                 sep='\t',
#                 header=False,
#                 index=False,
#                 quoting=csv.QUOTE_NONE,
#             )

#         except KeyError:
#             print(f'Empty source: {source} level: {level}')

Empty source: 1 level: 5


In [38]:
g.iloc[train_idx].loc[:, ['path', 'gt']]

,path,gt
76420,train_76420.jpg,f = \frac { A / n _ { 1 } } { B / n _ { 2 } } ...
14331,train_14331.jpg,= \lim _ { x \to 0 } \frac { 3 \left( \Delta x...
67369,train_67369.jpg,R = \left\{ z \leq - z _ { \frac { d } { 2 } }...
74534,train_74534.jpg,+ \lambda \left( r - r ^ { * } \right)
7145,train_07145.jpg,3 3 9 \times 2
...,...,...
93314,train_93314.jpg,f \left( x \right) = e ^ { a x }
76293,train_76293.jpg,+ \sum _ { t = 2 } ^ { n } \delta t \left( p ^...
42844,train_42844.jpg,V \subset U _ { k = 1 } ^ { \infty } I _ { k }
73151,train_73151.jpg,\max _ { i = 1 } ^ { n } \left| x _ { i } \rig...


In [34]:
train_len = int(len(g) * (1-0.2))
indexes = np.arange(len(g))
np.random.shuffle(indexes)
train_idx = indexes[:train_len]
val_idx = indexes[train_len:]

In [36]:
len(train_idx), len(val_idx)

(3565, 892)